# Tutorial about DSPy & LLMs

References:
* https://github.com/stanfordnlp/dspy/blob/main/skycamp2023.ipynb
* https://dspy-docs.vercel.app/docs/quick-start/minimal-example

In [ ]:
%pip install -q dspy-ai==2.4.12
%pip install -q huggingface-hub==0.23.0
%pip install -q transformers==4.40.2
%pip install -q accelerate==0.30.1
%pip install -q sentencepiece==0.2.0
%pip install -q datasets==2.19.1

In [4]:
import os
import yaml
from google.colab import drive
from getpass import getpass

drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Read YAML file
f_path = "/content/drive/MyDrive/GitHub/python-codebase/machine_learning/private_keys.yml"
with open(f_path, 'r') as stream:
    data_loaded = yaml.safe_load(stream)
os.environ['HF_API_TOKEN'] = data_loaded['HF_API_KEY']
os.environ['GITHUB_TOKEN'] = data_loaded['GITHUB_TOKEN']

## 1. Custom example

In [8]:
import dspy
from dspy.evaluate import Evaluate
from dspy.teleprompt import BootstrapFewShot, BootstrapFewShotWithRandomSearch, BootstrapFinetune

In [20]:
import pandas as pd
from io import BytesIO
df_train = pd.read_parquet("/content/drive/MyDrive/GitHub/python-codebase/machine_learning/datasets/gsm8k/train-00000-of-00001.parquet")
df_test = pd.read_parquet("/content/drive/MyDrive/GitHub/python-codebase/machine_learning/datasets/gsm8k/test-00000-of-00001.parquet")
display(df_train.head())
display(df_test.head())
print(len(df_train), len(df_test))

,question,answer
0,Natalia sold clips to 48 of her friends in Apr...,Natalia sold 48/2 = <<48/2=24>>24 clips in May...
1,Weng earns $12 an hour for babysitting. Yester...,Weng earns 12/60 = $<<12/60=0.2>>0.2 per minut...
2,Betty is saving money for a new wallet which c...,"In the beginning, Betty has only 100 / 2 = $<<..."
3,"Julie is reading a 120-page book. Yesterday, s...",Maila read 12 x 2 = <<12*2=24>>24 pages today....
4,James writes a 3-page letter to 2 different fr...,He writes each friend 3*2=<<3*2=6>>6 pages a w...


,question,answer
0,Janet’s ducks lay 16 eggs per day. She eats th...,Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eg...
1,A robe takes 2 bolts of blue fiber and half th...,It takes 2/2=<<2/2=1>>1 bolt of white fiber\nS...
2,Josh decides to try flipping a house. He buys...,The cost of the house and repairs came out to ...
3,James decides to run 3 sprints 3 times a week....,He sprints 3*3=<<3*3=9>>9 times\nSo he runs 9*...
4,"Every day, Wendi feeds each of her chickens th...","If each chicken eats 3 cups of feed per day, t..."


7473 1319


In [21]:
# Dataset (train)
nb_train = 10
train = [(row['question'], row['answer']) for i, row in df_train.head(nb_train).iterrows()]
print(train)
train = [dspy.Example(question=question, answer=answer).with_inputs('question') for question, answer in train]

[('Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?', 'Natalia sold 48/2 = <<48/2=24>>24 clips in May.\nNatalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.\n#### 72'), ('Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?', 'Weng earns 12/60 = $<<12/60=0.2>>0.2 per minute.\nWorking 50 minutes, she earned 0.2 x 50 = $<<0.2*50=10>>10.\n#### 10'), ('Betty is saving money for a new wallet which costs $100. Betty has only half of the money she needs. Her parents decided to give her $15 for that purpose, and her grandparents twice as much as her parents. How much more money does Betty need to buy the wallet?', "In the beginning, Betty has only 100 / 2 = $<<100/2=50>>50.\nBetty's grandparents gave her 15 * 2 = $<<15*2=30>>30.\nThis means, Betty needs 100 - 50 - 30 - 15 = $<<100-50-30-15=5>>5 more.\n#### 5"),

In [22]:
# Dataset (test)
nb_test = 10
test = [(row['question'], row['answer']) for i, row in df_test.head(nb_test).iterrows()]
print(test)
test = [dspy.Example(question=question, answer=answer).with_inputs('question') for question, answer in train]

[("Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?", 'Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.\nShe makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.\n#### 18'), ('A robe takes 2 bolts of blue fiber and half that much white fiber.  How many bolts in total does it take?', 'It takes 2/2=<<2/2=1>>1 bolt of white fiber\nSo the total amount of fabric is 2+1=<<2+1=3>>3 bolts of fabric\n#### 3'), ('Josh decides to try flipping a house.  He buys a house for $80,000 and then puts in $50,000 in repairs.  This increased the value of the house by 150%.  How much profit did he make?', 'The cost of the house and repairs came out to 80,000+50,000=$<<80000+50000=130000>>130,000\nHe increased the value of the house by 80,000*1.5=<<80000*1.5=120000

In [36]:
# Set up the LM.
model_name = 'mistralai/Mixtral-8x7B-Instruct-v0.1'
#llm_model = dspy.HFClientTGI(model="meta-llama/Llama-2-13b-chat-hf", port=[7140, 7141, 7142, 7143], max_tokens=250)
llm_model = dspy.HFClientVLLM(model=model_name, port=8080, url=f"https://api-inference.huggingface.co/models/{model_name}")

dspy.settings.configure(lm=llm_model)

TypeError: HFClientVLLM.__init__() missing 1 required positional argument: 'port'

In [35]:
## Zero-shot predictions

# Define a dspy.Predict module with the signature `question -> answer` (i.e., takes a question and outputs an answer).
predict = dspy.Predict('question -> answer')

# Use the module!
question_check = "Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?"
predict(question = question_check)

ConnectionError: HTTPConnectionPool(host='future-hgx-1', port=7142): Max retries exceeded with url: /generate (Caused by NameResolutionError("<urllib3.connection.HTTPConnection object at 0x7ee31567a8c0>: Failed to resolve 'future-hgx-1' ([Errno -2] Name or service not known)"))

In [25]:
class CoT(dspy.Module):  # let's define a new module
    def __init__(self):
        super().__init__()

        # here we declare the chain of thought sub-module, so we can later compile it (e.g., teach it a prompt)
        self.generate_answer = dspy.ChainOfThought('question -> answer')

    def forward(self, question):
        return self.generate_answer(question=question)  # here we use the module

In [ ]:
# Compile using train examples
metric_EM = dspy.evaluate.answer_exact_match

teleprompter = BootstrapFewShot(metric=metric_EM, max_bootstrapped_demos=2)
cot_compiled = teleprompter.compile(CoT(), trainset=train)

In [ ]:
cot_compiled(question_check)

In [ ]:
# Check history
llm_model.inspect_history(n=1)

In [ ]:
# Evaluate test set
NUM_THREADS = 32
evaluate_hotpot = Evaluate(devset=test, metric=metric_EM, num_threads=NUM_THREADS, display_progress=True, display_table=15)

In [ ]:
evaluate_hotpot(cot_compiled)